In [1]:
from tqdm import tqdm
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
import numpy as np

from data.ETFsZZR import ETFsZZR
from loss_functions.SharpeLoss import SharpeLoss
from models.TGNNPO import TGNNPO

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [22]:
# hyperparameters
device = torch.device('cpu')
epochs = 10
batch_size = 10
shuffle = True
drop_last = True
num_timesteps_in = 50
num_timesteps_out = 1
learning_rate = 1e-3
lossfn = SharpeLoss()

# load etfs dataset
loader = ETFsZZR()
dataset = loader.get_dataset(num_timesteps_in=num_timesteps_in, num_timesteps_out=num_timesteps_out)
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.7)

In [23]:
train_input = np.array(train_dataset.features)
train_target = np.array(train_dataset.targets)
train_x_tensor = torch.from_numpy(train_input).type(torch.FloatTensor).to(device)
train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor).to(device)
train_dataset_new = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

test_input = np.array(test_dataset.features)
test_target = np.array(test_dataset.targets)
test_x_tensor = torch.from_numpy(test_input).type(torch.FloatTensor).to(device)
test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor).to(device)
test_dataset_new = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset_new, batch_size=batch_size, shuffle=False, drop_last=drop_last)

# static graph
static_edge_index = next(iter(train_dataset)).edge_index.to(device)

In [24]:
# define model and optimizer
model = TGNNPO(node_features=2, periods=12, batch_size=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs + 1): 
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for steps, (X_batch, y_batch) in pbar:
        optimizer.zero_grad()
        
        # predict portfolio weights
        weights_pred = model(X_batch, static_edge_index)
  
        # sharpe ratio loss
        loss = lossfn(y_batch, weights_pred, ascent=True)
        pbar.set_description("Epoch: %d, sharpe (loss): %1.5f" % (epoch, loss.item() * -1))

        loss.backward()
        optimizer.step()    

Epoch: 10, sharpe (loss): 0.18720: 100%|██████████| 303/303 [00:04<00:00, 60.65it/s]


In [15]:
model.eval()
loss = 0
step = 0

# Store for analysis
weights = []
prices = []

pbar = tqdm(enumerate(test_loader), total=len(test_loader))
for steps, (X_batch, y_batch) in pbar:
    optimizer.zero_grad()

    # predict portfolio weights
    weights_pred = model(X_batch, static_edge_index)

    # sharpe ratio loss
    loss = lossfn(y_batch, weights_pred, ascent=True)
    pbar.set_description("Test: sharpe (loss): %1.5f" % (loss.item() * -1))

    # store predictions and true values
    prices.append(y_batch)
    weights.append(weights_pred)

    loss.backward()
    optimizer.step() 

Test: sharpe (loss): 0.03371: 100%|██████████| 131/131 [00:02<00:00, 48.90it/s] 
